In [1]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import csv
from sklearn.model_selection import train_test_split   # Import the train_test_split function
#import gzip

In [ ]:
# Open the first CSV file for reading
with open('archive (1)/output_file1.csv', 'r', newline='') as file1:
    reader1 = csv.reader(file1)
    data1 = list(reader1)

# Open the second CSV file for reading
with open('archive (1)/output_file2.csv', 'r', newline='') as file2:
    reader2 = csv.reader(file2)
    data2 = list(reader2)

# Combine the data from both files
combined_data = data1 + data2

# Create a new CSV file to write the combined data
with open('merged_data.csv', 'w', newline='') as output_file:
    writer = csv.writer(output_file)
    writer.writerows(combined_data)
df = pd.read_csv('merged_data.csv')
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df = df.sample(30000).drop('link', axis=1).reset_index(drop=True)
df.head(5)

In [ ]:
df['text'] = df['text'].str.lower().replace(r'^a-ZA-Z0-9','').replace(r'\n','',regex=True)







In [ ]:
df.head(5)







In [ ]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')
train_matrix = tfid.fit_transform(train_data['text'])

In [ ]:
stemmer = PorterStemmer()







In [ ]:
def token(x):
    token = nltk.word_tokenize(x)
    y = [stemmer.stem(i) for i in token]
    return " ".join(y)

In [ ]:
token("you are beautiful")

In [ ]:
df['text'].apply(lambda z: token(z))

In [ ]:
tfid = TfidfVectorizer(analyzer='word',stop_words='english')

In [ ]:
matrix = tfid.fit_transform(df['text'])

In [ ]:
similarity = cosine_similarity(matrix)

In [ ]:
df[df['song']=='A Child Is Born In Bethlehem'].index[0]

In [ ]:
def recommender(song):
    idx = df[df['song']==song].index[0]
    distance = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x:x[1])
    song= []
    for song_id in distance[1:5]:
        song.append(df.iloc[song_id[0]].song)
    return song

In [ ]:
recommender("Octopus's Garden")







In [ ]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))
pickle.dump(df, open('df.pkl', 'wb'))